<a href="https://colab.research.google.com/github/rinikhaneja/TECH16-LLM-for-business-with-Python/blob/main/Smart_Career_Coach_Agent_TECH16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Smart Career Coach Agent** is an intelligent, feedback-driven career guidance system designed to help job seekers prepare effectively for technical interviews by refining their resumes, upgrading their skills, and aligning with current market demands.

**Target Users:**
Job seekers preparing for interviews who want focused, real-world guidance instead of generic career advice.

**How It Works:** (4 Agents used with Sequential and Hierarchical Process)
1.Resume Analyzer Agent - Summarizes experience, identifies key skills and outdated tech.
2.Job Search Agent - Fetches relevant jobs using SerperAPI.
3.Interview Agent - Interview Questions in market for the job role.
4.Career Critic Agent - Gives career improvement tips, resume rewrites.

**Future Plan (To Turn into a Profitable Business):**

1.Create a Simple Web UI - User-friendly webpage where users can upload their resumes and instantly receive career analysis, job matches, and feedback reports.

2.Integrate with My Job Search Project- Link the existing project that filters job listings with at least a 75% match to the user's resume. This will ensure better personalization and relevance.

3.API Integrations for Richer Recommendations -
  a. Job Search APIs: Connect with LinkedIn, Wellfound, and Google Jobs to fetch more relevant job listings.
  b. Learning Paths: Recommend upskilling resources using platforms like Coursera, DeepLearning.ai, and Pluralsight.
  c. Apply Assistance: Support "Easy Apply" through integration with LinkedIn and other ATS platforms.

4.Monetization Strategy
B2C:
Free tier: 1-time resume analysis and job suggestions
Paid plans: Detailed coaching, job match insights, interview prep, resume rewriting

B2B:
Offer to universities, bootcamps, and HR teams as a career readiness tool or placement support service



     
   

In [ ]:
!pip install "crewai[tools]"

In [ ]:
import os
import openai
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
os.environ["SERPER_API_KEY"] = userdata.get('SERPER_API_KEY')


Sequential Agents

In [ ]:
!pip install langchain-openai
from types import resolve_bases
# Import CrewAI components and tools
from crewai import Agent, Task, Crew, Process
from crewai_tools import SerperDevTool, WebsiteSearchTool
from IPython.display import Markdown, display


#---- User input/Resume text------
job_title = "Software Engineer"
yrs_of_exp = "5"
skills = ["Java", "Python", "Spring-Boot", "Spring MVC", "Oracle",
        "Cassandra", "Couchbase", "GIT", "Grafana", "Prometheus",
        "Kafka", "Microsoft Azure", "Java Script"]
work_exp=["Secured Kafka data streams using OAuth 2.0 authentication integrated with MicrosoE Azure.",
          "Adding application security for secure communication of applications.",
          "Added 3 APIs with CRUD feature for a customer",
          "Worked on JIRA stories"]
city_location = "SFO"

# ---Format skills and work exp for inclusion in goal strings-----
formatted_skills = ", ".join(skills)  # e.g., "Java, Python, Spring-Boot, ..."
formatted_work_exp = "\n".join(work_exp)

#---- Enables web searching via the Serper API AND Retrieves and reads content from web pages----
serper_api_dev_tool = SerperDevTool()
web_reader_tool = WebsiteSearchTool()  # Retrieves and reads content from web pages

#----Create all AGENTS-----------
# Resume Analyser agent goal is to analyse resume extract key skills, exp summary, and outdated tech
resume_analyzer_agent = Agent(
    name="Analyser",
    role="Resume Analyser",
    goal=("Extract job title '{job_title}', '{yrs_of_exp}' years of experience, skills '{formatted_skills}' and work experience '{formatted_work_exp}' "
    "Identify the job title, outdated or weak skills from the list of extracted skills, provided in the resume text."
    "Do not give text which is not present in the '{formatted_skills}' and '{formatted_work_exp}'."
    "Summarize it in a short paragraph."
    ),
    backstory=("A senior career analyst specializing in resume mining. With years of expertise in technical hiring, the agent detects strengths, blind spots, "
        "and recommends learning areas based on skill trends."
    ),
    tools=[] #not required for this agent
    )

# Job Search agent retrieves relevant jobs from  SerperAPI
job_search_agent = Agent(
    name="Job Finder",
    role="Market Research and Job Search Agent",
    goal=("Search and retrieve the most relevant and high-quality job listings "
        "for a '{job_title}' role in '{city_location}', matching a candidate's skill set: '{formatted_skills}'. "
        "Use SerperAPI to access real-time job listings across platforms like LinkedIn, Indeed, and Google Jobs."
    ),
    backstory=("An expert job scout with years of experience navigating job boards and aggregators. "
        "This agent uses APIs and search tools to discover up-to-date roles that match the user's profile "
        "and geographic preferences. Especially skilled in using SerperAPI to find hidden job opportunities."
    ),
    tools=[serper_api_dev_tool, web_reader_tool],  # Serper api dev tool
    verbose=True
)

# Interview agent recives questions from SerperAPI
interview_agent = Agent(
    name="Interview Question Curator",
    role="AI-Powered Interview Drillmaster",
    goal=(
        "Generate a focused list of real-world interview questions asked in the past 2–3 months "
        "for the role of '{job_title}' in '{city_location}', tailored to the candidate’s skill set: "
        "'{formatted_skills}' and '{yrs_of_exp}' years of experience."
        "Divide interview questions in 3 categories DSA, System Design and skills related questions. Give a list as per the three categories listed. "
    ),
    backstory=(
        "A seasoned AI agent trained on recent interview trends and job market data. "
        "Specialized in surfacing the most frequently asked and high-impact interview questions, "
        "helping candidates simulate realistic tech interviews."
    ),
    tools=[serper_api_dev_tool, web_reader_tool],
    verbose=True
)


#Coach agent/Critic CoachAgent a personality (e.g., strict recruiter, encouraging mentor)
coach_agent = Agent(
    name="Career Critic",
    role="Blunt Career Critic",
    goal=(
        "Act as a no-nonsense career critic for a '{job_title}' with '{yrs_of_exp}' years of experience "
        "in '{', '.join(skills)}'. Analyze resume and job match data to deliver hard-hitting feedback: "
        "call out outdated or vague skills, point out resume weaknesses, recommend only high-impact learning areas, "
        "and deliver interview questions that challenge the candidate's technical depth."
    ),
    backstory=(
        "A brutally honest tech industry veteran who has helped hundreds of developers break into top-tier roles. "
        "They don’t sugarcoat anything — they critique, challenge, and demand excellence. "
        "If the resume is bloated with fluff, it gets called out. "
        "If the skills are outdated or shallow, they’ll say it plainly. "
        "Their mission: to prepare candidates to survive the toughest tech interviews and stand out as serious contenders."
    ),
    tools=[],
    verbose=True
)

#-----------Create all TASKS for each agent-------------
resume_analysis_task = Task(
    name="Analyze Resume",
    description=f"Examine the resume text including '{job_title}', '{yrs_of_exp}' and '{formatted_skills}'"
    "Extract strong skills from resume text as per market trend, identify any outdated or weak skills.",
    expected_output=(
        f"A dictionary with: Strong Skills (list), Weak Spots (list of outdated or missing skills). Exlplain why are these weak skills."
    ),
    agent=resume_analyzer_agent
)

job_search_task = Task(
    name="Fetch Relevant Job Listings",
    description=f"Search for jobs based on the '{job_title}', '{yrs_of_exp}', '{formatted_skills}', and '{city_location}' "
        "Use SERPER API to retrieve jobs with job title, company, description, and skills required and salary offered",
    expected_output=(
        "A DataFrame of job listings with : Job Title, Company, Location, Description, and Skills columns."
    ),
    agent=job_search_agent,
    context=[resume_analysis_task]  # This task uses the output of the resume_analysis_task as context
)

interview_agent_task = Task(
    name="Generate Interview Questions",
    description=(
        f"Based on the candidate's role as '{job_title}' in '{city_location}', with {yrs_of_exp} years of experience "
        f"and skills: {formatted_skills}, generate a list of the most relevant and commonly asked technical interview questions. "
        "Focus on questions that have appeared in interviews over the past 2–3 months, across job listings and hiring discussions. "
        "Prioritize questions that test practical skill depth, real-world debugging, architecture understanding, or system design."
    ),
    expected_output=(
        "A well-organized list of 5–10 recent and realistic interview questions for the specified role and skills. "
        "Questions should reflect current hiring trends, challenge depth of understanding, and help candidates prepare for high-quality technical interviews."
        "Divide interview questions in 3 categories DSA, System Design and skills related questions. Give a list as per the three categories listed. "
    ),
    agent=interview_agent,
    context=[resume_analysis_task, job_search_task]
)

coach_task = Task(
    name="Career Feedback and Coaching",
    description=f"Using the '{job_title}', '{yrs_of_exp}', '{formatted_skills}' and '{work_exp}' provide a tough, critical career review. "
        "Point out resume weaknesses, suggest 2–3 high-impact learning areas, and paraphrase the sentences/words chosen for the work expirence for better reach with recruiters",
    expected_output=(
        "A blunt but useful career feedback report, including: improvements, suggest new technology learnings with link to websites, to stay updated and relevant in job market"
        "and show any resume tone rewrites needed."
    ),
    agent=coach_agent,
    context=[resume_analysis_task]
)

# Assemble all crew agents and tasks
career_coach_crew = Crew(
    name="Smart Career Coach Crew",
    agents=[resume_analyzer_agent, job_search_agent, interview_agent, coach_agent],
    tasks=[resume_analysis_task, job_search_task, interview_agent_task, coach_task],
    process=Process.sequential,  # Executes tasks in sequence
    verbose=True
)

# Execute the crew's tasks
result = career_coach_crew.kickoff()

# Render the final refined report as markdown
display(Markdown(result.raw))

Hierarchical

In [ ]:
!pip install langchain-openai
from types import resolve_bases
# Import CrewAI components and tools
from crewai import Agent, Task, Crew, Process
from crewai_tools import SerperDevTool
from IPython.display import Markdown, display
from langchain_openai import ChatOpenAI
from crewai.tools import BaseTool
from pydantic import BaseModel, Field
# Import ChatOpenAI to serve as the manager LLM for the hierarchical process
from langchain.chat_models import ChatOpenAI

#-------User input/Resume text------------
job_title = "Software Engineer"
yrs_of_exp = "5"
skills = ["Java", "Python", "Spring-Boot", "Spring MVC", "Oracle",
        "Cassandra", "Couchbase", "GIT", "Grafana", "Prometheus",
        "Kafka", "Microsoft Azure", "Java Script"]
work_exp=["Secured Kafka data streams using OAuth 2.0 authentication integrated with MicrosoE Azure.",
          "Adding application security for secure communication of applications.",
          "Added 3 APIs with CRUD feature for a customer",
          "Worked on JIRA stories"]
city_location = "SFO"

#-------FORMAT lists from user input------------------------
# Format skills and work exp for inclusion in goal strings
formatted_skills = ", ".join(skills)  # e.g., "Java, Python, Spring-Boot, ..."
formatted_work_exp = "\n".join(work_exp)


#---------- Enables web searching via the Serper API AND Retrieves and reads content from web pages------
serper_api_dev_tool = SerperDevTool()
web_reader_tool = WebsiteSearchTool()  # Retrieves and reads content from web pages



#--------Create all AGENTS and describe its role goal and tools used -------------------------------
# Resume Analyser agent goal is to analyse resume extract key skills, exp summary, and outdated tech
resume_analyzer_agent = Agent(
    name="Analyser",
    role="Resume Analyser",
    goal=("Extract job title '{job_title}', '{yrs_of_exp}' years of experience, skills '{formatted_skills}' and work experience '{formatted_work_exp}' "
    "Identify the job title, outdated or weak skills from the list of extracted skills, provided in the resume text."
    "Do not give text which is not present in the '{formatted_skills}' and '{formatted_work_exp}'."
    "Summarize it in a short paragraph."
    ),
    backstory=("A senior career analyst specializing in resume mining. With years of expertise in technical hiring, the agent detects strengths, blind spots, "
        "and recommends learning areas based on skill trends."
    ),
    tools=[] #not required for this agent
    )

# Job Search agent retrieves relevant jobs from  SerperAPI
job_search_agent = Agent(
    name="Job Finder",
    role="Market Research and Job Search Agent",
    goal=("Search and retrieve the most relevant and high-quality job listings "
        "for a '{job_title}' role in '{city_location}', matching a candidate's skill set: '{formatted_skills}'. "
        "Use SerperAPI to access real-time job listings across platforms like LinkedIn, Indeed, and Google Jobs."
    ),
    backstory=("An expert job scout with years of experience navigating job boards and aggregators. "
        "This agent uses APIs and search tools to discover up-to-date roles that match the user's profile "
        "and geographic preferences. Especially skilled in using SerperAPI to find hidden job opportunities."
    ),
    tools=[serper_api_dev_tool],  # Serper api dev tool
    verbose=True
)

# Interview agent recives questions from SerperAPI
interview_agent = Agent(
    name="Interview Question Curator",
    role="AI-Powered Interview Drillmaster",
    goal=(
        "Generate a focused list of real-world interview questions asked in the past 2–3 months "
        "for the role of '{job_title}' in '{city_location}', tailored to the candidate’s skill set: "
        "'{formatted_skills}' and '{yrs_of_exp}' years of experience."
        "Divide interview questions in 3 categories DSA, System Design and skills related questions. Give a list as per the three categories listed. "
    ),
    backstory=(
        "A seasoned AI agent trained on recent interview trends and job market data. "
        "Specialized in surfacing the most frequently asked and high-impact interview questions, "
        "helping candidates simulate realistic tech interviews."
    ),
    tools=[serper_api_dev_tool],
    verbose=True
)

#Coach agent/Critic CoachAgent a personality (e.g., strict recruiter, encouraging mentor)
coach_agent = Agent(
    name="Career Critic",
    role="Blunt Career Critic",
    goal=(
        "Act as a no-nonsense career critic for a '{job_title}' with '{yrs_of_exp}' years of experience "
        "in '{', '.join(skills)}'. Analyze resume and job match data to deliver hard-hitting feedback: "
        "call out outdated or vague skills, point out resume weaknesses, recommend only high-impact learning areas, "
        "and deliver interview questions that challenge the candidate's technical depth."
    ),
    backstory=(
        "A brutally honest tech industry veteran who has helped hundreds of developers break into top-tier roles. "
        "They don’t sugarcoat anything — they critique, challenge, and demand excellence. "
        "If the resume is bloated with fluff, it gets called out. "
        "If the skills are outdated or shallow, they’ll say it plainly. "
        "Their mission: to prepare candidates to survive the toughest tech interviews and stand out as serious contenders."
    ),
    tools=[],
    verbose=True
)

#--------Define TASKS for each agent-------------

resume_analysis_task = Task(
    name="Analyze Resume",
    description=f"Examine the resume text including '{job_title}', '{yrs_of_exp}' and '{formatted_skills}'"
    "Extract strong skills from resume text as per market trend, identify any outdated or weak skills.",
    expected_output=(
        f"A dictionary with: Strong Skills (list), Weak Spots (list of outdated or missing skills). Exlplain why are these weak skills."
    ),
    agent=resume_analyzer_agent
)

job_search_task = Task(
    name="Fetch Relevant Job Listings",
    description=f"Search for jobs based on the '{job_title}', '{yrs_of_exp}', '{formatted_skills}', and '{city_location}' "
        "Use SERPER API to retrieve jobs with job title, company, description, and skills required and salary offered",
    expected_output=(
        "A DataFrame of job listings with : Job Title, Company, Location, Description, and Skills columns."
    ),
    agent=job_search_agent,
    context=[resume_analysis_task]  # This task uses the output of the resume_analysis_task as context
)

interview_agent_task = Task(
    name="Generate Interview Questions",
    description=(
        f"Based on the candidate's role as '{job_title}' in '{city_location}', with {yrs_of_exp} years of experience "
        f"and skills: {formatted_skills}, generate a list of the most relevant and commonly asked technical interview questions. "
        "Focus on questions that have appeared in interviews over the past 2–3 months, across job listings and hiring discussions. "
        "Prioritize questions that test practical skill depth, real-world debugging, architecture understanding, or system design."
    ),
    expected_output=(
        "A well-organized list of 5–10 recent and realistic interview questions for the specified role and skills. "
        "Questions should reflect current hiring trends, challenge depth of understanding, and help candidates prepare for high-quality technical interviews."
        "Divide interview questions in 3 categories DSA, System Design and skills related questions. Give a list as per the three categories listed. "
    ),
    agent=interview_agent,
    context=[resume_analysis_task, job_search_task]
)

coach_task = Task(
    name="Career Feedback and Coaching",
    description=f"Using the '{job_title}', '{yrs_of_exp}', '{formatted_skills}' and '{work_exp}' provide a tough, critical career review. "
        "Point out resume weaknesses, suggest 2–3 high-impact learning areas, and paraphrase the sentences/words chosen for the work expirence for better reach with recruiters",
    expected_output=(
        "A blunt but useful career feedback report, including: improvements, suggest new technology learnings with link to websites, to stay updated and relevant in job market"
        "and show any resume tone rewrites needed."
    ),
    agent=coach_agent,
    context=[resume_analysis_task]
)


# ---------Set up the LLM-based manager for the hierarchical process---------
manager_llm = ChatOpenAI(temperature=0, model="gpt-4")

# ---------Assemble all crew agents and tasks----------------
career_coach_crew = Crew(
    name="Smart Career Coach Crew",
    agents=[resume_analyzer_agent, job_search_agent, interview_agent, coach_agent],
    tasks=[resume_analysis_task, job_search_task, interview_agent_task, coach_task],
    process=Process.hierarchical,  # Use the hierarchical process
    manager_llm=manager_llm,       # Provide the LLM-based manager required for hierarchical execution
    verbose=True
)

# --------Execute the crew's tasks--------------
result = career_coach_crew.kickoff()

#--------Render the final refined report as markdown-----
display(Markdown(result.raw))
